# Install pyspark library

In [67]:
!pip install pyspark

In [68]:
#import pyspark library
import pyspark

In [69]:
#import spark session library
from pyspark.sql import SparkSession

In [70]:
# Create SparkSession object
spark = SparkSession.builder \
                    .master('local[*]') \
                    .appName('linear_regression') \
                    .getOrCreate()

# Create DataFrame

In [71]:
#To create dataframe form External datasets
df = spark.read.option("header", "true").csv("/Users/krishnapratap/Desktop/partation/data/*")

In [72]:
df.show()

+----+-------+-----+----------+---------+----------+-------------+---------+-------+-------+---------+---------------+------------------+------------------+------+--------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+---------------+---------+-------------+-------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+--------------+-----------------+-------+-------+--------+-------------+------------+------------+--------+-------------+-----------------+------------+-------------+---------------+------------------+--------------+--------------------+-----------+-----------+-----------+-------------+----------------+------------+--------------+----------------+-------------+-----------+-----------+-------------+--------------

22/04/26 14:41:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Year, Quarter, Month, DayofMonth, DayOfWeek, FlightDate, UniqueCarrier, AirlineID, Carrier, TailNum, FlightNum, OriginAirportID, OriginAirportSeqID, OriginCityMarketID, Origin, OriginCityName, OriginState, OriginStateFips, OriginStateName, OriginWac, DestAirportID, DestAirportSeqID, DestCityMarketID, Dest, DestCityName, DestState, DestStateFips, DestStateName, DestWac, CRSDepTime, DepTime, DepDelay, DepDelayMinutes, DepDel15, DepartureDelayGroups, DepTimeBlk, TaxiOut, WheelsOff, WheelsOn, TaxiIn, CRSArrTime, ArrTime, ArrDelay, ArrDelayMinutes, ArrDel15, ArrivalDelayGroups, ArrTimeBlk, Cancelled, CancellationCode, Diverted, CRSElapsedTime, ActualElapsedTime, AirTime, Flights, Distance, DistanceGroup, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay, FirstDepTime, TotalAddGTime, LongestAddGTime, DivAirportLandings, DivReachedDest, DivActualElapsedTime, DivArrDelay, DivDistan

# Data Preparation

In [73]:
# create new dataframe as per required columns for prediction
AirlineDF = df.select("Origin","Dest","AirTime","Distance")

In [74]:
#cache data in-memory for fast read write operation
AirlineDF.cache()

DataFrame[Origin: string, Dest: string, AirTime: string, Distance: string]

In [75]:
#check data in new dataframe
AirlineDF.show()

+------+----+-------+--------+
|Origin|Dest|AirTime|Distance|
+------+----+-------+--------+
|   JFK| LAX| 338.00| 2475.00|
|   JFK| LAX| 349.00| 2475.00|
|   JFK| LAX| 370.00| 2475.00|
|   JFK| LAX| 350.00| 2475.00|
|   JFK| LAX| 335.00| 2475.00|
|   JFK| LAX| 336.00| 2475.00|
|   JFK| LAX| 380.00| 2475.00|
|   JFK| LAX| 359.00| 2475.00|
|   JFK| LAX| 368.00| 2475.00|
|   JFK| LAX| 356.00| 2475.00|
|   JFK| LAX| 353.00| 2475.00|
|   JFK| LAX| 332.00| 2475.00|
|   JFK| LAX| 339.00| 2475.00|
|   JFK| LAX| 339.00| 2475.00|
|   JFK| LAX| 335.00| 2475.00|
|   JFK| LAX| 340.00| 2475.00|
|   JFK| LAX| 327.00| 2475.00|
|   JFK| LAX| 308.00| 2475.00|
|   JFK| LAX| 315.00| 2475.00|
|   JFK| LAX| 323.00| 2475.00|
+------+----+-------+--------+
only showing top 20 rows



In [76]:
#check data types of each columns
AirlineDF.printSchema()

root
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- Distance: string (nullable = true)



# We need to change data types of columns

In [77]:
#import sql library for data types 
from pyspark.sql.types import IntegerType

In [78]:
AirlineDF = AirlineDF.withColumn("Distance", AirlineDF["Distance"].cast(IntegerType()))

In [79]:
AirlineDF = AirlineDF.withColumn("AirTime", AirlineDF["AirTime"].cast(IntegerType()))

In [80]:
#check data types of each columns
AirlineDF.printSchema()

root
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- AirTime: integer (nullable = true)
 |-- Distance: integer (nullable = true)



# Check is there any null values in DataFrame 

In [81]:
#import libray for sql function col
from pyspark.sql.functions import col

In [82]:
#check null values in each columns
print(AirlineDF.where(col("Origin").isNull()).count())
print(AirlineDF.where(col("Dest").isNull()).count())
print(AirlineDF.where(col("AirTime").isNull()).count())
print(AirlineDF.where(col("Distance").isNull()).count())

0
0
183789
0


# we need to handel null values 

In [83]:
#now drop row as correspondence to null values
AirlineDF = AirlineDF.dropna(subset=["AirTime"])

In [84]:
#check again
print(AirlineDF.where(col("Origin").isNull()).count())
print(AirlineDF.where(col("Dest").isNull()).count())
print(AirlineDF.where(col("AirTime").isNull()).count())
print(AirlineDF.where(col("Distance").isNull()).count())

0
0


0
0


# we need convert distance miles into km

In [85]:
from pyspark.sql.functions import round

# Convert 'mile' to 'km' 
AirlineDF = AirlineDF.withColumn('Distance', round(AirlineDF.Distance * 1.60934, 0))

AirlineDF.show()

+------+----+-------+--------+
|Origin|Dest|AirTime|Distance|
+------+----+-------+--------+
|   JFK| LAX|    338|  3983.0|
|   JFK| LAX|    349|  3983.0|
|   JFK| LAX|    370|  3983.0|
|   JFK| LAX|    350|  3983.0|
|   JFK| LAX|    335|  3983.0|
|   JFK| LAX|    336|  3983.0|
|   JFK| LAX|    380|  3983.0|
|   JFK| LAX|    359|  3983.0|
|   JFK| LAX|    368|  3983.0|
|   JFK| LAX|    356|  3983.0|
|   JFK| LAX|    353|  3983.0|
|   JFK| LAX|    332|  3983.0|
|   JFK| LAX|    339|  3983.0|
|   JFK| LAX|    339|  3983.0|
|   JFK| LAX|    335|  3983.0|
|   JFK| LAX|    340|  3983.0|
|   JFK| LAX|    327|  3983.0|
|   JFK| LAX|    308|  3983.0|
|   JFK| LAX|    315|  3983.0|
|   JFK| LAX|    323|  3983.0|
+------+----+-------+--------+
only showing top 20 rows



In [86]:
AirlineDF.printSchema()

root
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- AirTime: integer (nullable = true)
 |-- Distance: double (nullable = true)



## Verctorize the features

In [87]:
from pyspark.ml.feature import *

In [88]:
from pyspark.ml.feature import VectorAssembler

In [89]:
vectorizer = VectorAssembler()
vectorizer.setInputCols(["Distance"])
vectorizer.setOutputCol("features")

df_vect = vectorizer.transform(AirlineDF)

In [90]:
df_vect.show()

+------+----+-------+--------+--------+
|Origin|Dest|AirTime|Distance|features|
+------+----+-------+--------+--------+
|   JFK| LAX|    338|  3983.0|[3983.0]|
|   JFK| LAX|    349|  3983.0|[3983.0]|
|   JFK| LAX|    370|  3983.0|[3983.0]|
|   JFK| LAX|    350|  3983.0|[3983.0]|
|   JFK| LAX|    335|  3983.0|[3983.0]|
|   JFK| LAX|    336|  3983.0|[3983.0]|
|   JFK| LAX|    380|  3983.0|[3983.0]|
|   JFK| LAX|    359|  3983.0|[3983.0]|
|   JFK| LAX|    368|  3983.0|[3983.0]|
|   JFK| LAX|    356|  3983.0|[3983.0]|
|   JFK| LAX|    353|  3983.0|[3983.0]|
|   JFK| LAX|    332|  3983.0|[3983.0]|
|   JFK| LAX|    339|  3983.0|[3983.0]|
|   JFK| LAX|    339|  3983.0|[3983.0]|
|   JFK| LAX|    335|  3983.0|[3983.0]|
|   JFK| LAX|    340|  3983.0|[3983.0]|
|   JFK| LAX|    327|  3983.0|[3983.0]|
|   JFK| LAX|    308|  3983.0|[3983.0]|
|   JFK| LAX|    315|  3983.0|[3983.0]|
|   JFK| LAX|    323|  3983.0|[3983.0]|
+------+----+-------+--------+--------+
only showing top 20 rows



In [91]:
print(vectorizer.explainParams())

handleInvalid: How to handle invalid data (NULL and NaN values). Options are 'skip' (filter out rows with invalid data), 'error' (throw an error), or 'keep' (return relevant number of NaN in the output). Column lengths are taken from the size of ML Attribute Group, which can be set using `VectorSizeHint` in a pipeline before `VectorAssembler`. Column lengths can also be inferred from first rows of the data since it is safe to do so but only in case of 'error' or 'skip'). (default: error)
inputCols: input column names. (current: ['Distance'])
outputCol: output column name. (default: VectorAssembler_402a1e91be99__output, current: features)


# Train Test data Splitting

In [92]:
flights_train, flights_test = df_vect.randomSplit([0.8, 0.2])

In [93]:
flights_train.show()

+------+----+-------+--------+--------+
|Origin|Dest|AirTime|Distance|features|
+------+----+-------+--------+--------+
|   ABE| CLT|     70|   774.0| [774.0]|
|   ABE| CLT|     72|   774.0| [774.0]|
|   ABE| CLT|     74|   774.0| [774.0]|
|   ABE| CLT|     76|   774.0| [774.0]|
|   ABE| CLT|     77|   774.0| [774.0]|
|   ABE| CLT|     77|   774.0| [774.0]|
|   ABE| CLT|     77|   774.0| [774.0]|
|   ABE| CLT|     79|   774.0| [774.0]|
|   ABE| CLT|     79|   774.0| [774.0]|
|   ABE| CLT|     82|   774.0| [774.0]|
|   ABE| CLT|     82|   774.0| [774.0]|
|   ABE| CLT|     83|   774.0| [774.0]|
|   ABE| CLT|     83|   774.0| [774.0]|
|   ABE| CLT|     84|   774.0| [774.0]|
|   ABE| CLT|     84|   774.0| [774.0]|
|   ABE| CLT|     84|   774.0| [774.0]|
|   ABE| CLT|     84|   774.0| [774.0]|
|   ABE| CLT|     86|   774.0| [774.0]|
|   ABE| CLT|     87|   774.0| [774.0]|
|   ABE| CLT|     87|   774.0| [774.0]|
+------+----+-------+--------+--------+
only showing top 20 rows



# Regression Model Training

In [94]:
from pyspark.ml.regression import LinearRegression

In [95]:
lr = LinearRegression()
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber (default: 1.35)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
loss: The loss function to be optimized. Supported options: squaredError, huber. (default: squaredError)
maxBlockSizeInMB: maximum memory in MB for stacking input data into blocks. Data is stacked within partitions. If more than remaining data size in a partition then it is adjusted to the data size. Default 0.0 represents choosing optimal value, depends on specific algorithm. Must be >= 0. (default: 0.0)
maxIter: max number of iterations (>= 0). (default: 100)
predic

In [96]:
lr.setLabelCol("AirTime")
lr.setFeaturesCol("features")
model = lr.fit(flights_train)

22/04/26 14:42:15 WARN Instrumentation: [b44e815e] regParam is zero, which might cause numerical instability and overfitting.


In [97]:
type(model)

pyspark.ml.regression.LinearRegressionModel

# View model summary

In [98]:
print("R2:", model.summary.r2)
print("Intercept: ", model.intercept, "Coefficients", model.coefficients)

R2: 0.9581868297028161
Intercept:  18.17434950779186 Coefficients [0.07330927341040765]


# Model Testing 

In [99]:
df_pred = model.transform(flights_test)
df_pred.show()

+------+----+-------+--------+--------+-----------------+
|Origin|Dest|AirTime|Distance|features|       prediction|
+------+----+-------+--------+--------+-----------------+
|   ABE| CLT|     77|   774.0| [774.0]|74.91572712744738|
|   ABE| CLT|     78|   774.0| [774.0]|74.91572712744738|
|   ABE| CLT|     87|   774.0| [774.0]|74.91572712744738|
|   ABE| PHL|     17|    89.0|  [89.0]|24.69887484131814|
|   ABE| PHL|     18|    89.0|  [89.0]|24.69887484131814|
|   ABE| PHL|     21|    89.0|  [89.0]|24.69887484131814|
|   ABE| PHL|     23|    89.0|  [89.0]|24.69887484131814|
|   ABE| PHL|     24|    89.0|  [89.0]|24.69887484131814|
|   ABE| PHL|     25|    89.0|  [89.0]|24.69887484131814|
|   ABE| PHL|     28|    89.0|  [89.0]|24.69887484131814|
|   ABQ| AMA|     36|   446.0| [446.0]|50.87028544883367|
|   ABQ| AMA|     36|   446.0| [446.0]|50.87028544883367|
|   ABQ| AMA|     37|   446.0| [446.0]|50.87028544883367|
|   ABQ| AMA|     38|   446.0| [446.0]|50.87028544883367|
|   ABQ| AMA| 